## Understand & Map Silver tables → Customers

GOLD DIMENSION: CUSTOMERS
This dimension combines customer data from CRM and ERP systems
to provide a unified, analytics-ready customer view.


In [0]:
%sql
USE CATALOG analytics;
USE SCHEMA silver;


## SQL SELECT + JOIN (preview i dimension-it)

In [0]:
%sql
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    c.gender,
    c.marital_status,
    e.birth_date,
    l.country,
    c.cst_create_date AS customer_created_date
FROM silver.crm_customers c
LEFT JOIN silver.erp_customers e
    ON CAST(c.customer_id AS STRING) = CAST(e.customer_id AS STRING)
LEFT JOIN silver.erp_customers_location l
    ON CAST(c.customer_id AS STRING) = CAST(l.customer_id AS STRING);


## GOLD DIMENSION: CUSTOMERS

In [0]:
%sql
-- This table provides a unified, analytics-ready customer view
-- by combining CRM customer attributes with ERP demographic and location data.

CREATE OR REPLACE TABLE gold.dim_customers AS
SELECT
    c.customer_id,                      -- Business customer identifier (STRING)
    c.first_name,                       -- Customer first name from CRM
    c.last_name,                        -- Customer last name from CRM
    c.gender,                           -- Gender (CRM)
    c.marital_status,                   -- Marital status (CRM)
    e.birth_date,                       -- Birth date from ERP system
    l.country,                          -- Customer country from ERP location data
    c.cst_create_date AS customer_created_date  -- Customer creation date in CRM
FROM silver.crm_customers c
LEFT JOIN silver.erp_customers e
    -- Cast IDs to STRING to avoid type mismatch between systems
    ON CAST(c.customer_id AS STRING) = CAST(e.customer_id AS STRING)
LEFT JOIN silver.erp_customers_location l
    -- Join customer location data
    ON CAST(c.customer_id AS STRING) = CAST(l.customer_id AS STRING);


In [0]:
%sql
SELECT COUNT(*) FROM gold.dim_customers;


## Control for duplicates

In [0]:
%sql
SELECT customer_id, COUNT(*)
FROM gold.dim_customers
GROUP BY customer_id
HAVING COUNT(*) > 1;


In [0]:
%sql
COMMENT ON TABLE analytics.gold.dim_customers IS
'Customer dimension table containing one row per customer.
Provides standardized customer attributes for analytics and reporting.';
